# 0 - Initializations

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import csv

from pylab import rcParams
import seaborn as sns
import math
import os
from datetime import datetime,  timedelta

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
 
from matplotlib import pyplot

from solarpy import irradiance_on_plane
from solarpy import solar_panel
from numpy import array

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Input, Dense, GRU, Embedding, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, RepeatVector, Lambda, TimeDistributed, Embedding, TimeDistributed, BatchNormalization, Reshape, concatenate, Permute
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.backend import square, mean
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

#%run Database.ipynb
%run AuxFunctions.ipynb

plt.rcParams.update({'figure.figsize':(20,7), 'figure.dpi':300})

Using TensorFlow backend.


# 1 - Data Preparation

Cololect building data

In [2]:
df_pv, df_consumption = collectFromDatabase(False)

Build entire dataset

In [3]:
dfinal = getData(local = True)

# 2 - Build dataframes

In [4]:
a1, a2, a3, a4, a5, v1, v2, v3, v4, t5, d6 = getDatasets(dfinal)

# 3.1 - Define the train and validation sets

In [ ]:
model_name = 'F2-LSTM-256-4-CI' 

In [ ]:
dt = a4.copy()
dataset = '4'

final_features = features2()
dt = dt[final_features].copy()

In [ ]:
dt_val = v4.copy()

final_features = features2()
dt_val = dt_val[final_features].copy()

In [ ]:
dt_test = t5.copy()

final_features = features2()
dt_test = dt_test[final_features].copy()

In [ ]:
train = dt.values
val = dt_val.values
test = dt_test.values

# 3.2 - Build supervised learning sequences

In [ ]:
n_pre = 60
n_post = 15

dX, dY = [], []
for i in range(len(train)-n_pre-n_post):
    dX.append(train[i:i+n_pre])
trainX = np.array(dX)


for i in range(len(train)-n_pre-n_post):

    ar = np.array(([row[7] for row in train[i+n_pre:i+n_pre+n_post]][4], [row[7] for row in train[i+n_pre:i+n_pre+n_post]][9], [row[7] for row in train[i+n_pre:i+n_pre+n_post]][14]))
    B = ar.reshape(-1, len(ar))
    dY.append(B)
    
trainY = np.array(dY)


vX, vY = [], []
for i in range(len(val)-n_pre-n_post):
    vX.append(val[i:i+n_pre])
valX = np.array(vX)

for i in range(len(val)-n_pre-n_post):

    ar = np.array(([row[7] for row in val[i+n_pre:i+n_pre+n_post]][4], [row[7] for row in val[i+n_pre:i+n_pre+n_post]][9], [row[7] for row in val[i+n_pre:i+n_pre+n_post]][14]))
    B = ar.reshape(-1, len(ar))
    vY.append(B)

valY = np.array(vY)

In [ ]:
trainX_original = trainX.copy()
trainY_original = trainY.copy()
valX_original = valX.copy()
valY_original = valY.copy()

In [ ]:
trainY = trainY.reshape(trainY.shape[0], trainY.shape[2])
valY = valY.reshape(valY.shape[0], valY.shape[2])

In [ ]:
trainX.shape, trainY.shape, valX.shape, valY.shape

In [ ]:
tX, tY = [], []
for i in range(len(test)-n_pre-n_post):
    tX.append(test[i:i+n_pre])
testX = np.array(tX)

for i in range(len(test)-n_pre-n_post):

    ar = np.array(([row[7] for row in test[i+n_pre:i+n_pre+n_post]][4], [row[7] for row in test[i+n_pre:i+n_pre+n_post]][9], [row[7] for row in test[i+n_pre:i+n_pre+n_post]][14]))
    B = ar.reshape(-1, len(ar))
    tY.append(B)

testY = np.array(tY)

In [ ]:
testX_original = testX.copy()
testY_original = testY.copy()

In [ ]:
testY = testY.reshape(testY.shape[0], testY.shape[2])

In [ ]:
trainX.shape, trainY.shape, valX.shape, valY.shape, testX.shape, testY.shape

# 3.3 - Scale the data

In [ ]:
scalersX = {}

for i in range(trainX.shape[2]):
    scalersX[i] = MinMaxScaler(feature_range=(0, 1))
    trainX[:, :, i] = scalersX[i].fit_transform(trainX[:, :, i]) 

for i in range(valX.shape[2]):
    valX[:, :, i] = scalersX[i].transform(valX[:, :, i]) 
    
for i in range(testX.shape[2]):
    testX[:, :, i] = scalersX[i].transform(testX[:, :, i]) 

In [ ]:
scalerY = MinMaxScaler(feature_range=(0, 1))
trainY = scalerY.fit_transform(trainY) 
valY = scalerY.transform(valY)
testY = scalerY.transform(testY)

In [ ]:
testY[0,0]

# 3.4 - Create and train the model

Load existing model

In [ ]:
# load json and create model
json_file = open('AZURE/files3/outputs/models/'+'ED-GRU-256-1-60', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights('AZURE/files3/outputs/models/'+'ED-GRU-256-1-60' + '.h5')
print("Loaded model from disk")

In [ ]:
loaded_model.compile(loss='mse', optimizer='adam', metrics = ['mae', 'mse', tf.keras.metrics.RootMeanSquaredError()])

Create new model

In [ ]:
print('creating model...')
model = create_model_vanilla(steps_before = n_pre, feature_count = 8, units = 256, layer = 'LSTM', intervals = True)
callbacks, path_checkpoint = callbacksFunction(model_name, dataset)

In [ ]:
history = train_model(model, trainX, trainY, valX, valY, 512, 100, callbacks, True)

In [ ]:
try:
    model.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

In [ ]:
n_epochs = len(history.history['loss'])
times = callbacks[4].times 
Time_Epoch = Average(times) 
Total_Time = sum(times) 

Train_mae = min(history.history['mae'])
Train_mse = min(history.history['mse'])
Train_rmse = min(history.history['root_mean_squared_error'])

Validation_mae = min(history.history['val_mae'])
Validation_mse = min(history.history['val_mse'])
Validation_rmse = min(history.history['val_root_mean_squared_error'])

# 3.5 - Perform predictions

Single prediction (without intervals)

In [ ]:
predict = loaded_model.predict(testX)
predict_original = predict.copy()
nan_array = np.empty((n_pre - 1))
nan_array.fill(np.nan)
nan_array2 = np.empty(n_post)
nan_array2.fill(np.nan)
ind = np.arange(n_pre + n_post)
predict = scalerY.inverse_transform(predict_original)

N=20 predictions (with intervals)

In [ ]:
nan_array = np.empty((n_pre - 1))
nan_array.fill(np.nan)
nan_array2 = np.empty(n_post)
nan_array2.fill(np.nan)
ind = np.arange(n_pre + n_post)

GlobalPredictions = []
GlobalPredictions_normalized = []

for j in range (0,20):
    
    print(j, end= ' ')
    predict_ = model.predict(testX)
    predict_original_ = predict_.copy()

    predict = scalerY.inverse_transform(predict_)
    
    GlobalPredictions.append(predict)
    GlobalPredictions_normalized.append(predict_)
    
GlobalPredictions = np.array(GlobalPredictions)
GlobalPredictions_normalized = np.array(GlobalPredictions_normalized)

# 3.6 - Plot the predictions 

Without intervals

In [ ]:
aux = np.full([valX_original.shape[0],1,15], np.nan)

aux2 = np.full([predict.shape[0],1,15], np.nan)

fig, ax = plt.subplots(figsize=(10, 3.5))
for i in range(200, valX.shape[0], valX.shape[0]):

    aux[i, 0, 4] = valY_original[i, 0, 0]
    aux[i, 0, 9] = valY_original[i, 0, 1]
    aux[i, 0, 14] = valY_original[i, 0, 2]
    
    aux2[i, 0, 4] = predict[i, 0]
    aux2[i, 0, 9] = predict[i, 1]
    aux2[i, 0, 14] = predict[i, 2]
    
    forecasts_original = np.concatenate((nan_array, valX_original[i, -1:, 7], aux2[i, 0, :]))
    ground_truth = np.concatenate((nan_array, valX_original[i, -1:, 7], aux[i, 0, :]))
    network_input = np.concatenate((valX_original[i, :, 7], nan_array2))

    plt.xticks(rotation=45)
    
    SMALLER_SIZE = 18
    SMALL_SIZE = 19
    MEDIUM_SIZE = 23
    BIGGER_SIZE = 25

    #plt.('test title', fontsize=BIGGER_SIZE)
    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALLER_SIZE)  # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    
    
    ax.plot(ind, ground_truth, 'r-o', markersize=10, markeredgecolor='black', linewidth=2, label='Av. Power')
    
    ax.plot(ind, forecasts_original, 'go--', markersize=10, linewidth=2, marker='h', markerfacecolor='lightgreen', \
             markeredgewidth=2, label='Forecast')
            
    ax.plot(ind[40:], network_input[40:], '-o', markersize=10, markeredgecolor='black', linewidth=2, label='Av. Power')
    
 
    
    plt.ticklabel_format(axis='y', style='sci', scilimits=(5,5))
    plt.xlabel('Date')
    plt.ylabel('Available Power [W]')
    plt.title('Model 1 - Available Power - Forecast')
    plt.legend(loc='lower left')
    #plt.savefig('Images/' + model_name, bbox_inches = 'tight')
    plt.savefig('Images/' + model_name + '_op2', bbox_inches = 'tight')

With Intervals

In [ ]:
mean = np.mean(GlobalPredictions, axis=0)

ci_1 = 0.80
lower_lim_1 = np.quantile(GlobalPredictions, 0.5-ci_1/2, axis=0)
upper_lim_1 = np.quantile(GlobalPredictions, 0.5+ci_1/2, axis=0)

ci_2 = 0.95
lower_lim_2 = np.quantile(GlobalPredictions, 0.5-ci_2/2, axis=0)
upper_lim_2 = np.quantile(GlobalPredictions, 0.5+ci_2/2, axis=0)


aux = np.full([testX_original.shape[0],1,15], np.nan)

aux2 = np.full([mean.shape[0],1,15], np.nan)

aux3 = np.full([lower_lim_1.shape[0],1,15], np.nan)

aux4 = np.full([upper_lim_1.shape[0],1,15], np.nan)

aux5 = np.full([lower_lim_2.shape[0],1,15], np.nan)

aux6 = np.full([upper_lim_2.shape[0],1,15], np.nan)

fig, ax = plt.subplots(figsize=(10, 3.5))
for i in range(635, testX.shape[0], testX.shape[0]):
    
    aux[i, 0, 4] = testY_original[i, 0, 0]
    aux[i, 0, 9] = testY_original[i, 0, 1]
    aux[i, 0, 14] = testY_original[i, 0, 2]
    
    aux2[i, 0, 4] = mean[i, 0]
    aux2[i, 0, 9] = mean[i, 1]
    aux2[i, 0, 14] = mean[i, 2]
    
    aux3[i, 0, 4] = lower_lim_1[i, 0]
    aux3[i, 0, 9] = lower_lim_1[i, 1]
    aux3[i, 0, 14] = lower_lim_1[i, 2]
    
    aux4[i, 0, 4] = upper_lim_1[i, 0]
    aux4[i, 0, 9] = upper_lim_1[i, 1]
    aux4[i, 0, 14] = upper_lim_1[i, 2]
    
    aux5[i, 0, 4] = lower_lim_2[i, 0]
    aux5[i, 0, 9] = lower_lim_2[i, 1]
    aux5[i, 0, 14] = lower_lim_2[i, 2]
    
    aux6[i, 0, 4] = upper_lim_2[i, 0]
    aux6[i, 0, 9] = upper_lim_2[i, 1]
    aux6[i, 0, 14] = upper_lim_2[i, 2]
    
    forecasts = np.concatenate((nan_array, testX_original[i, -1:, 7], aux2[i, 0, :]))
    
    lower_lim_1 = np.concatenate((nan_array, testX_original[i, -1:, 7], aux3[i, 0, :]))
    upper_lim_1 = np.concatenate((nan_array, testX_original[i, -1:, 7], aux4[i, 0, :]))
    lower_lim_2 = np.concatenate((nan_array, testX_original[i, -1:, 7], aux5[i, 0, :]))
    upper_lim_2 = np.concatenate((nan_array, testX_original[i, -1:, 7], aux6[i, 0, :]))
    
    ground_truth = np.concatenate((nan_array, testX_original[i, -1:, 7], aux[i, 0, :]))
    network_input = np.concatenate((testX_original[i, :, 7], nan_array2))

    plt.xticks(rotation=45)
    
    SMALLER_SIZE = 18
    SMALL_SIZE = 19
    MEDIUM_SIZE = 23
    BIGGER_SIZE = 25

    #plt.('test title', fontsize=BIGGER_SIZE)
    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALLER_SIZE)  # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


    
    plt.fill_between(ind, lower_lim_2, upper_lim_2, color='orange', linewidth=2, label = str(int(ci_2 *100)) + '% CI')
        
    ax.plot(ind, lower_lim_2, '-o', color='orange', markersize=20, marker='_')
    ax.plot(ind, upper_lim_2, '-o', color='orange', markersize=20, marker='_')
    
    ax.plot(ind, ground_truth, 'r-o', markersize=10, markeredgecolor='black', linewidth=2, label='Av. Power')
    
    ax.plot(ind, forecasts, 'go--', markersize=10, linewidth=2, marker='h', markerfacecolor='lightgreen', \
             markeredgewidth=2, label='Forecast')
            
    ax.plot(ind[40:], network_input[40:], '-o', markersize=10, markeredgecolor='black', linewidth=2, label='Av. Power')
    
    plt.ticklabel_format(axis='y', style='sci', scilimits=(5,5))
    plt.xlabel('Date')
    plt.ylabel('Available Power [W]')
    plt.title('Model 2 - Available Power - Forecast')
    plt.legend(loc='lower left')
    plt.savefig('Images/' + model_name , bbox_inches = 'tight')
    #plt.savefig('Images/' + model_name + '_op2', bbox_inches = 'tight')

# 3.7 - Get arrays with the predictions of only 5, 10 and 5 minutes ahead

Without Intervals

In [ ]:
predicted_normalized = []
original_normalized = []

predicted = []
original = []

aux = np.full([valY_original.shape[0],1,15], np.nan)
aux5 = np.full([valY.shape[0],1,15], np.nan)
aux1 = np.full([predict.shape[0],1,15], np.nan)
aux2 = np.full([predict.shape[0],1,15], np.nan)


for i in range(0, valX.shape[0]):

    aux[i, 0, 4] = valY_original[i, 0, 0]
    aux[i, 0, 9] = valY_original[i, 0, 1]
    aux[i, 0, 14] = valY_original[i, 0, 2]

    aux1[i, 0, 4] = predict_original[i, 0]
    aux1[i, 0, 9] = predict_original[i, 1]
    aux1[i, 0, 14] = predict_original[i, 2]

    aux2[i, 0, 4] = predict[i, 0]
    aux2[i, 0, 9] = predict[i, 1]
    aux2[i, 0, 14] = predict[i, 2]

    aux5[i, 0, 4] = valY[i, 0]
    aux5[i, 0, 9] = valY[i, 1]
    aux5[i, 0, 14] = valY[i, 2]

    forecasts_normalized = np.concatenate((nan_array, valX[i, -1:, 7], aux1[i, 0, :]))
    forecasts = np.concatenate((nan_array, valX_original[i, -1:, 7], aux2[i, 0, :]))

    ground_truth_normalized = np.concatenate((nan_array, valX[i, -1:, 7], aux5[i, 0, :]))
    ground_truth = np.concatenate((nan_array, valX_original[i, -1:, 7], aux[i, 0, :]))

    predicted_normalized.append((forecasts_normalized[n_pre+4], forecasts_normalized[n_pre+9], forecasts_normalized[n_pre+14]))
    predicted.append((forecasts[n_pre+4], forecasts[n_pre+9], forecasts[n_pre+14]))

    original_normalized.append((ground_truth_normalized[n_pre+4], ground_truth_normalized[n_pre+9], ground_truth_normalized[n_pre+14]))
    original.append((ground_truth[n_pre+4], ground_truth[n_pre+9], ground_truth[n_pre+14]))

With Intervals

In [ ]:
predicted_normalized = []
predicted_mean_normalized = []
original_normalized = []

predicted = []
predicted_mean = []
original = []

mean = np.mean(GlobalPredictions, axis=0)

mean_normalized = np.mean(GlobalPredictions_normalized, axis=0)

aux = np.full([testY_original.shape[0],1,15], np.nan)

aux5 = np.full([testY.shape[0],1,15], np.nan)

aux1 = np.full([predict.shape[0],1,15], np.nan)

aux2 = np.full([predict.shape[0],1,15], np.nan)

aux3 = np.full([mean.shape[0],1,15], np.nan)

aux4 = np.full([mean_normalized.shape[0],1,15], np.nan)


for j in range (0, testX.shape[0]):
    for i in range(j, testX.shape[0], testX.shape[0]):
        
        aux[i, 0, 4] = testY_original[i, 0, 0]
        aux[i, 0, 9] = testY_original[i, 0, 1]
        aux[i, 0, 14] = testY_original[i, 0, 2]
        
        aux3[i, 0, 4] = mean[i, 0]
        aux3[i, 0, 9] = mean[i, 1]
        aux3[i, 0, 14] = mean[i, 2]
        
        aux4[i, 0, 4] = mean_normalized[i, 0]
        aux4[i, 0, 9] = mean_normalized[i, 1]
        aux4[i, 0, 14] = mean_normalized[i, 2]
        
        aux5[i, 0, 4] = testY[i, 0]
        aux5[i, 0, 9] = testY[i, 1]
        aux5[i, 0, 14] = testY[i, 2]
        
        forecasts_mean_normalized = np.concatenate((nan_array, testX[i, -1:, 7], aux4[i, 0, :]))
        forecasts_mean = np.concatenate((nan_array, testX_original[i, -1:, 7], aux3[i, 0, :]))
        
        ground_truth_normalized = np.concatenate((nan_array, testX[i, -1:, 7], aux5[i, 0, :]))
        ground_truth = np.concatenate((nan_array, testX_original[i, -1:, 7], aux[i, 0, :]))
        
        predicted_mean_normalized.append((forecasts_mean_normalized[n_pre+4], forecasts_mean_normalized[n_pre+9], forecasts_mean_normalized[n_pre+14]))
        predicted_mean.append((forecasts_mean[n_pre+4], forecasts_mean[n_pre+9], forecasts_mean[n_pre+14]))
        
        original_normalized.append((ground_truth_normalized[n_pre+4], ground_truth_normalized[n_pre+9], ground_truth_normalized[n_pre+14]))
        original.append((ground_truth[n_pre+4], ground_truth[n_pre+9], ground_truth[n_pre+14]))

# 3.8 - Compute the validation errors

In [ ]:
validation_rmse_5_normalized, validation_rmse_10_normalized, validation_rmse_15_normalized, \
validation_mse_5_normalized, validation_mse_10_normalized, validation_mse_15_normalized, \
validation_mae_5_normalized, validation_mae_10_normalized, validation_mae_15_normalized = \
printValidationErrorsNormalized(original_normalized, predicted_normalized)

In [ ]:
validation_rmse_5, validation_rmse_10, validation_rmse_15, \
validation_mse_5, validation_mse_10, validation_mse_15, \
validation_mae_5, validation_mae_10, validation_mae_15 = \
printValidationErrors(original, predicted)

# 3.9 - Compute the test errors

In [ ]:
test_rmse_5, test_rmse_10, test_rmse_15,\
test_mse_5, test_mse_10, test_mse_15,\
test_mae_5, test_mae_10, test_mae_15 = printTestErrors(original, predicted_mean)

In [ ]:
test_rmse_5_n, test_rmse_10_n, test_rmse_15_n,\
test_mse_5_n, test_mse_10_n, test_mse_15_n,\
test_mae_5_n, test_mae_10_n, test_mae_15_n = printTestErrorsNormalized(original_normalized, predicted_mean_normalized)

# 3.10 - Plot the training and the validation error evolution

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(history.history['loss'],label='Train MSE')
plt.plot(history.history['val_loss'],label='Validation MSE')
plt.title('Model Loss')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(loc='best')
plt.savefig('Images/' + model_name + '_Loss_' + dataset)
plt.show()

# 3.11 - Save the results

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open('models/' + model_name, "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights('models/' + model_name + '.h5')
print("Saved model to disk")

In [ ]:
result = model.evaluate(x=testX, y=testY)
print("loss (test-set):", result)

In [ ]:
with open('Test_' + dataset + '.csv', 'a', newline='') as file:
    fieldnames = ['Model', 
                  'Test_MSE_n', 'Test_RMSE_n', 'Test_MAE_n',                  
                  'Test_5_RMSE_n', 'Test_10_RMSE_n', 'Test_15_RMSE_n', 
                  'Test_5_MSE_n', 'Test_10_MSE_n', 'Test_15_MSE_n', 
                  'Test_5_MAE_n', 'Test_10_MAE_n', 'Test_15_MAE_n',
                  
                  'Test_5_RMSE', 'Test_10_RMSE', 'Test_15_RMSE', 
                  'Test_5_MSE', 'Test_10_MSE', 'Test_15_MSE', 
                  'Test_5_MAE', 'Test_10_MAE', 'Test_15_MAE']
    
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()

with open('Test_' + dataset + '.csv', 'a', newline='') as file:
    fieldnames = ['Model',
                  'Test_MSE_n', 'Test_RMSE_n', 'Test_MAE_n',                  
                  'Test_5_RMSE_n', 'Test_10_RMSE_n', 'Test_15_RMSE_n', 
                  'Test_5_MSE_n', 'Test_10_MSE_n', 'Test_15_MSE_n', 
                  'Test_5_MAE_n', 'Test_10_MAE_n', 'Test_15_MAE_n',
                
                  'Test_5_RMSE', 'Test_10_RMSE', 'Test_15_RMSE', 
                  'Test_5_MSE', 'Test_10_MSE', 'Test_15_MSE', 
                  'Test_5_MAE', 'Test_10_MAE', 'Test_15_MAE']
    
    writer = csv.DictWriter(file, fieldnames=fieldnames)
       
    writer.writerow({'Model': model_name, 
                     
                     'Test_MSE_n': result[2], 
                     'Test_RMSE_n': result[3],
                     'Test_MAE_n': result[1], 
                     
                     'Test_5_RMSE_n': test_rmse_5_n,
                     'Test_10_RMSE_n': test_rmse_10_n,
                     'Test_15_RMSE_n': test_rmse_15_n, 
                     'Test_5_MSE_n': test_mse_5_n,
                     'Test_10_MSE_n': test_mse_10_n,
                     'Test_15_MSE_n': test_mse_15_n, 
                     'Test_5_MAE_n': test_mae_5_n,
                     'Test_10_MAE_n': test_mae_10_n,
                     'Test_15_MAE_n': test_mae_15_n,
                                     
                     'Test_5_RMSE': test_rmse_5,
                     'Test_10_RMSE': test_rmse_10,
                     'Test_15_RMSE': test_rmse_15, 
                     'Test_5_MSE': test_mse_5,
                     'Test_10_MSE': test_mse_10,
                     'Test_15_MSE': test_mse_15, 
                     'Test_5_MAE': test_mae_5,
                     'Test_10_MAE': test_mae_10,
                     'Test_15_MAE': test_mae_15                 
                                        })

In [ ]:
with open('ValidationProcess_' + dataset + '.csv', 'a', newline='') as file:
    fieldnames = ['Model', 'Time', 'Epochs', 'Time_Epoch',
                  'Train_MSE', 'Train_RMSE', 'Train_MAE', 
                  'Validation_MSE', 'Validation_RMSE', 'Validation_MAE',                  
                  'Validation_5_RMSE', 'Validation_10_RMSE', 'Validation_15_RMSE', 
                  'Validation_5_MSE', 'Validation_10_MSE', 'Validation_15_MSE', 
                  'Validation_5_MAE', 'Validation_10_MAE', 'Validation_15_MAE']
    
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()

with open('ValidationProcess_' + dataset + '.csv', 'a', newline='') as file:
    fieldnames = ['Model', 'Time', 'Epochs', 'Time_Epoch',
                  'Train_MSE', 'Train_RMSE', 'Train_MAE',
                  'Validation_MSE', 'Validation_RMSE', 'Validation_MAE',                  
                  
                  'Validation_5_RMSE', 'Validation_10_RMSE', 'Validation_15_RMSE', 
                  'Validation_5_MSE', 'Validation_10_MSE', 'Validation_15_MSE', 
                  'Validation_5_MAE', 'Validation_10_MAE', 'Validation_15_MAE']
    
    writer = csv.DictWriter(file, fieldnames=fieldnames)
       
    writer.writerow({'Model': model_name, 
                     'Time': Total_Time, 
                     'Epochs': n_epochs, 
                     'Time_Epoch': Time_Epoch, 
                     
                     'Train_MSE': Train_mse,
                     'Train_RMSE': Train_rmse,
                     'Train_MAE': Train_mae, 
                     'Validation_MSE': Validation_mse, 
                     'Validation_RMSE': Validation_rmse, 
                     'Validation_MAE':Validation_mae , 

                  
                     'Validation_5_RMSE': validation_rmse_5_normalized, 
                     'Validation_10_RMSE': validation_rmse_10_normalized, 
                     'Validation_15_RMSE': validation_rmse_15_normalized,
                     'Validation_5_MSE': validation_mse_5_normalized, 
                     'Validation_10_MSE': validation_mse_10_normalized, 
                     'Validation_15_MSE': validation_mse_15_normalized,
                     'Validation_5_MAE': validation_mae_5_normalized, 
                     'Validation_10_MAE': validation_mae_10_normalized, 
                     'Validation_15_MAE': validation_mae_15_normalized,})


